# ChatBot for AI-Mini Project

### Importing Libraries

In [80]:
import random
import json
import pickle
from statistics import mode
import numpy as np
import nltk
import tensorflow as tf
import pandas as pd

from tensorflow.keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

### Initializing LemmaTokenizer

In [81]:
lemmatizer = WordNetLemmatizer()

### Loading Dataset

In [82]:
intents = json.loads(open('intents.json').read())

### Initializing Variables

In [83]:
words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ","]

### Populating the Data in the Dictionary

In [84]:
train_data = pd.read_csv('train.csv')
train_data

,Prompt,Class
0,Hi,greeting
1,How are you,greeting
2,Is anyone there?,greeting
3,Hello,greeting
4,Good day,greeting
...,...,...
158,identify as?,gender
159,Identity?,gender
160,your identity?,gender
161,are you a man?,gender


In [85]:
for row in train_data.itertuples():
    print(row)
    print(row[1])
    print(row[2])
    word_list = nltk.word_tokenize(row[1])
    print("Word List: {}".format(word_list))
    words.extend(word_list)

    documents.append(((word_list), row[2]))
    # print(documents)
    if(row[2] not in classes):
        classes.append(row[2])
words = [lemmatizer.lemmatize(word.lower())
         for word in words if word not in ignore_letters]
words = sorted(list(set(words)))

Pandas(Index=0, Prompt='Hi', Class='greeting')
Hi
greeting
Word List: ['Hi']
Pandas(Index=1, Prompt='How are you', Class='greeting')
How are you
greeting
Word List: ['How', 'are', 'you']
Pandas(Index=2, Prompt='Is anyone there?', Class='greeting')
Is anyone there?
greeting
Word List: ['Is', 'anyone', 'there', '?']
Pandas(Index=3, Prompt='Hello', Class='greeting')
Hello
greeting
Word List: ['Hello']
Pandas(Index=4, Prompt='Good day', Class='greeting')
Good day
greeting
Word List: ['Good', 'day']
Pandas(Index=5, Prompt='Hello there', Class='greeting')
Hello there
greeting
Word List: ['Hello', 'there']
Pandas(Index=6, Prompt='Good morning', Class='greeting')
Good morning
greeting
Word List: ['Good', 'morning']
Pandas(Index=7, Prompt='Good evening', Class='greeting')
Good evening
greeting
Word List: ['Good', 'evening']
Pandas(Index=8, Prompt='how is it going', Class='greeting')
how is it going
greeting
Word List: ['how', 'is', 'it', 'going']
Pandas(Index=9, Prompt='greetings', Class='greet

In [86]:
# for intent in intents["intents"]:
#     print("Intent: {}".format(intent))
#     for pattern in intent["patterns"]:
#         word_list = nltk.word_tokenize(pattern)
#         print("Word List: {}".format(word_list))
#         words.extend(word_list)

#         documents.append(((word_list), intent["tag"]))
#         # print(documents)
#         if(intent["tag"] not in classes):
#             classes.append(intent["tag"])

# words = [lemmatizer.lemmatize(word.lower())
#          for word in words if word not in ignore_letters]
# words = sorted(list(set(words)))

### Saving the Data for Future Use

In [87]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

### Creating the data Dataset

In [88]:
data = []
output_empty = [0] * len(classes)
for document in documents:
    print("Document: {}".format(document))
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(
        word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    data.append([bag, output_row])

Document: (['Hi'], 'greeting')
Document: (['How', 'are', 'you'], 'greeting')
Document: (['Is', 'anyone', 'there', '?'], 'greeting')
Document: (['Hello'], 'greeting')
Document: (['Good', 'day'], 'greeting')
Document: (['Hello', 'there'], 'greeting')
Document: (['Good', 'morning'], 'greeting')
Document: (['Good', 'evening'], 'greeting')
Document: (['how', 'is', 'it', 'going'], 'greeting')
Document: (['greetings'], 'greeting')
Document: (['Tell', 'me', 'something'], 'greeting')
Document: (['What', 'is', 'your', 'name', '?'], 'name')
Document: (['What', 'should', 'I', 'call', 'you', '?'], 'name')
Document: (['What', 'do', 'you', 'go', 'by', '?'], 'name')
Document: (['What', "'s", 'your', 'name', '?'], 'name')
Document: (['What', 'should', 'I', 'call', 'you', '?'], 'name')
Document: (['Can', 'you', 'introduce', 'yourself', '?'], 'name')
Document: (['name'], 'name')
Document: (['alias'], 'name')
Document: (['nickname'], 'name')
Document: (['what', 'do', 'the', 'common', 'folk', 'call', 'you'

In [89]:
# import train_test_split
random.shuffle(data)
data = np.array(data, dtype=object)

train_x = list(data[:int(len(data) * 0.8), 0])
train_y = list(data[:int(len(data) * 0.8), 1])
test_x = list(data[int(len(data) * 0.8):, 0])
test_y = list(data[int(len(data) * 0.8):, 1])

### Creating the Model

In [90]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 128)               18944     
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 13)                845       
                                                                 
Total params: 28,045
Trainable params: 28,045
Non-trainable params: 0
_________________________________________________________________


### Compiling and data the Model

In [91]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy",
              optimizer=sgd, metrics=["accuracy"])
history = model.fit(np.array(train_x), np.array(
    train_y), epochs=300, batch_size=5, verbose=1)

Epoch 1/300
26/26 [==============================] - 0s 1ms/step - loss: 2.5616 - accuracy: 0.0615
Epoch 2/300
26/26 [==============================] - 0s 1ms/step - loss: 2.4424 - accuracy: 0.2154
Epoch 3/300
26/26 [==============================] - 0s 1ms/step - loss: 2.2977 - accuracy: 0.2385
Epoch 4/300
26/26 [==============================] - 0s 1ms/step - loss: 2.1621 - accuracy: 0.3000
Epoch 5/300
26/26 [==============================] - 0s 1ms/step - loss: 2.0017 - accuracy: 0.3769
Epoch 6/300
26/26 [==============================] - 0s 1ms/step - loss: 1.9264 - accuracy: 0.3692
Epoch 7/300
26/26 [==============================] - 0s 1ms/step - loss: 1.8181 - accuracy: 0.4538
Epoch 8/300
26/26 [==============================] - 0s 1ms/step - loss: 1.5884 - accuracy: 0.5077
Epoch 9/300
26/26 [==============================] - 0s 1ms/step - loss: 1.5743 - accuracy: 0.5538
Epoch 10/300
26/26 [==============================] - 0s 1ms/step - loss: 1.4184 - accuracy: 0.5538
Epoch 11/

### Exporting the Model

In [92]:
model.save("chatbot_model.h5", history)

In [93]:
# find accuracy
from sklearn.metrics import classification_report
y_pred = model.predict(test_x)
# compare to test_y
# accuracy = classification_report(test_y, y_pred)

tp = 0
tn = 0
fp = 0
fn = 0
for i in range(len(y_pred)):
    if (np.argmax(y_pred[i]) == np.argmax(test_y[i])):
        tp += 1
    else:
        fp += 1

In [94]:
acc = tp / (tp + fp)    
acc

0.696969696969697